In [2]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(
    model='llama3.2:1b',
    temperature=0
)

In [3]:
resp = llm.invoke("How are you today?")
print(resp.content)

I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm here and ready to help you with any questions or topics you'd like to discuss. How about you? How's your day going so far?


In [4]:
from langchain_core.prompts import PromptTemplate

RAG_PROMPT_TEMPLATE = """\
Answer the questions based on the context provided. If the context doesn't have the answer, say that you
don't know the answer.

User Query:
{query}

Context:
{context}
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

In [5]:
rag_chain = rag_prompt | llm

In [6]:
resp = rag_chain.invoke({"query" : "Who old is Carl?", "context" : "Carl is a sweet dude, he's 40."})
print(resp.content)

I can help with that. Based on the context, it seems that Carl is 40 years old.


In [7]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

In [48]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

FILE_PATH = "./eu_ai_act.pdf"

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
Loader = PyMuPDFLoader
loader = Loader(FILE_PATH)
documents = loader.load()
docs = text_splitter.split_documents(documents)
# for i, doc in enumerate(docs):
#     doc.metadata["source"] = f"source_{i}"

In [49]:
from langchain_qdrant import QdrantVectorStore

url = "http://localhost:6334"
qdrant = QdrantVectorStore.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=True,
    collection_name="EU_AI_Act",
)

In [15]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
url = "http://localhost:6333"
client = QdrantClient(url=url)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="EU_AI_Act",
    embedding=embeddings,
)